# Sudoku-4 : Recuit Simule (Python)

**Niveau** : Metaheuristique | **Duree** : ~20 min | **Prerequis** : Sudoku-0 Environment

## Navigation

| << Precedent | [Index](README.md) | Suivant >> |
|-------------|---------------------|-----------|
| [Sudoku-3-Genetic-Python](Sudoku-3-Genetic-Python.ipynb) | | [Sudoku-5-PSO-Python](Sudoku-5-PSO-Python.ipynb) |

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Formuler la resolution de Sudoku comme un probleme d'optimisation
2. Definir une fonction d'energie comptant les violations de contraintes
3. Construire un voisinage par echange de cellules
4. Implementer un solveur par recuit simule avec la bibliotheque `simanneal`
5. Analyser les forces et limites du recuit simule pour le Sudoku

---

Ce notebook implemente un solveur de Sudoku utilisant le recuit simule (Simulated Annealing) en Python.
C'est l'equivalent Python du notebook C# `Sudoku-4-SimulatedAnnealing-Csharp.ipynb`.

## Introduction

Le **recuit simule** (Simulated Annealing, SA) est une metaheuristique inspiree du processus metallurgique de recuit : un metal est chauffe puis refroidi lentement pour atteindre un etat cristallin optimal.

### Principe

- **Etat** : Une grille entierement remplie (potentiellement avec des erreurs)
- **Energie** : Mesure le nombre de violations de contraintes
- **Temperature** : Controle l'acceptation de mouvements degradants
- **Refroidissement** : Reduit progressivement la temperature

### Critere d'acceptation de Metropolis

$$P(\text{accepter}) = \begin{cases} 1 & \text{si } \Delta E \leq 0 \\ e^{-\Delta E / T} & \text{si } \Delta E > 0 \end{cases}$$

ou $\Delta E = E(\text{voisin}) - E(\text{courant})$ et $T$ est la temperature courante.

## Installation

```bash
pip install simanneal numpy matplotlib
```

In [ ]:
# Imports
import numpy as np
import time
import random
from typing import List, Tuple, Optional
import matplotlib.pyplot as plt

try:
    from simanneal import Annealer
    print(f"simanneal importe avec succes")
except ImportError:
    print("simanneal non installe. Executez: pip install simanneal")
    raise

In [ ]:
# Configuration du chemin vers les puzzles
import os
from pathlib import Path

NOTEBOOK_DIR = Path(r"D:\Dev\CoursIA\MyIA.AI.Notebooks\Sudoku")
PUZZLES_DIR = NOTEBOOK_DIR / "Puzzles"

if PUZZLES_DIR.exists():
    print(f"Dossier Puzzles: {PUZZLES_DIR}")
else:
    print(f"ATTENTION: Dossier Puzzles non trouve a {PUZZLES_DIR}")
    PUZZLES_DIR = Path(os.getcwd()) / "Puzzles"

## 1. Classe SudokuGrid

Representation d'une grille de Sudoku avec methodes pour calculer l'energie et generer des voisins.

In [ ]:
class SudokuGrid:
    """Representation d'une grille de Sudoku 9x9."""
    
    def __init__(self, grid: Optional[np.ndarray] = None):
        if grid is None:
            self.cells = np.zeros((9, 9), dtype=int)
        else:
            self.cells = grid.copy()
    
    @classmethod
    def from_string(cls, s: str) -> 'SudokuGrid':
        s = s.replace('.', '0').replace(' ', '').replace('\n', '')
        if len(s) != 81:
            raise ValueError(f"La chaine doit avoir 81 caracteres")
        grid = cls()
        grid.cells = np.array([int(c) for c in s], dtype=int).reshape(9, 9)
        return grid
    
    def clone(self) -> 'SudokuGrid':
        return SudokuGrid(self.cells.copy())
    
    def to_string(self) -> str:
        return ''.join(str(self.cells[r, c]) for r in range(9) for c in range(9))
    
    def __str__(self) -> str:
        lines = []
        for r in range(9):
            if r > 0 and r % 3 == 0:
                lines.append('-' * 21)
            row_str = ''
            for c in range(9):
                if c > 0 and c % 3 == 0:
                    row_str += '| '
                val = self.cells[r, c]
                row_str += (str(val) if val != 0 else '.') + ' '
            lines.append(row_str)
        return '\n'.join(lines)

def load_puzzles(filepath: str, max_puzzles: int = None) -> List[str]:
    puzzles = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if len(line) >= 81:
                puzzles.append(line[:81])
                if max_puzzles and len(puzzles) >= max_puzzles:
                    break
    return puzzles

# Charger les puzzles
easy_puzzles = load_puzzles(str(PUZZLES_DIR / 'Sudoku_Easy51.txt'), max_puzzles=5)
print(f"Puzzles charges: {len(easy_puzzles)}")

test_grid = SudokuGrid.from_string(easy_puzzles[0])
print("\nGrille de test:")
print(test_grid)

## 2. Fonction d'Energie

L'energie compte le nombre de violations de contraintes dans les colonnes et les blocs 3x3. Les lignes sont garanties valides par construction (permutations).

In [ ]:
def compute_energy(grid: np.ndarray) -> int:
    """Calcule l'energie d'une grille (nombre de doublons colonnes + blocs)."""
    energy = 0
    
    # Doublons dans les colonnes
    for col in range(9):
        values = grid[:, col]
        counts = np.bincount(values[values > 0], minlength=10)
        energy += np.sum(counts[1:] - 1)  # -1 car la valeur presente compte 1
    
    # Doublons dans les blocs 3x3
    for box_row in range(3):
        for box_col in range(3):
            block = grid[box_row*3:(box_row+1)*3, box_col*3:(box_col+1)*3].flatten()
            counts = np.bincount(block[block > 0], minlength=10)
            energy += np.sum(counts[1:] - 1)
    
    return energy

print("Fonction compute_energy definie.")
print(f"Energie de la grille de test: {compute_energy(test_grid.cells)}")

### Interpretation : Fonction d'energie

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Energie = 0 | Solution valide | Aucun doublon colonnes/blocs |
| Energie > 0 | Grille invalide | Nombre total de violations |

L'objectif est de reduire l'energie a 0 par echanges successifs.

## 3. Initialisation de la Grille

Chaque ligne est initialisee comme une permutation de 1-9, en respectant les cellules fixes.

In [ ]:
def initialize_grid(puzzle: SudokuGrid, rng: random.Random) -> Tuple[np.ndarray, np.ndarray]:
    """Initialise une grille avec permutations par ligne."""
    grid = puzzle.cells.copy()
    is_fixed = puzzle.cells > 0
    
    for row in range(9):
        # Identifier les valeurs fixes et manquantes
        fixed_values = set(grid[row, grid[row] > 0])
        missing_values = list(set(range(1, 10)) - fixed_values)
        empty_cols = [c for c in range(9) if grid[row, c] == 0]
        
        # Melanger les valeurs manquantes
        rng.shuffle(missing_values)
        
        # Remplir les cellules vides
        for i, col in enumerate(empty_cols):
            grid[row, col] = missing_values[i]
    
    return grid, is_fixed

rng = random.Random(42)
init_grid, is_fixed = initialize_grid(test_grid, rng)

print("Grille initialisee (chaque ligne est une permutation de 1-9):")
print(SudokuGrid(init_grid))
print(f"\nEnergie initiale: {compute_energy(init_grid)}")
print(f"Objectif: energie = 0")

## 4. Voisinage par Echange

Le voisinage est defini par l'echange de deux cellules non fixes dans une meme ligne. Cela preserve la propriete de permutation de la ligne.

In [ ]:
def generate_neighbor(grid: np.ndarray, is_fixed: np.ndarray, rng: random.Random) -> Tuple[int, int, int]:
    """Genere un voisin par echange de deux cellules non fixes dans une ligne.
    
    Returns:
        (row, col1, col2): Ligne et colonnes echangees
    """
    # Choisir une ligne avec au moins 2 cellules non fixes
    while True:
        row = rng.randint(0, 8)
        free_cols = [c for c in range(9) if not is_fixed[row, c]]
        if len(free_cols) >= 2:
            break
    
    # Choisir deux cellules distinctes
    col1, col2 = rng.sample(free_cols, 2)
    
    # Effectuer l'echange
    grid[row, col1], grid[row, col2] = grid[row, col2], grid[row, col1]
    
    return row, col1, col2

def undo_swap(grid: np.ndarray, row: int, col1: int, col2: int):
    """Annule un echange."""
    grid[row, col1], grid[row, col2] = grid[row, col2], grid[row, col1]

# Demonstration
test_init = init_grid.copy()
energy_before = compute_energy(test_init)
swap_row, swap_col1, swap_col2 = generate_neighbor(test_init, is_fixed, rng)
energy_after = compute_energy(test_init)

print(f"Echange effectue : ligne {swap_row}, colonnes {swap_col1} <-> {swap_col2}")
print(f"Energie avant : {energy_before}, Energie apres : {energy_after}, Delta : {energy_after - energy_before}")

## 5. Solveur par Recuit Simule avec simanneal

La bibliotheque `simanneal` fournit un cadre pour implementer facilement le recuit simule.

In [ ]:
class SudokuAnnealer(Annealer):
    """Solveur de Sudoku par recuit simule utilisant simanneal."""
    
    def __init__(self, puzzle: SudokuGrid):
        # Etat initial : grille avec permutations par ligne
        rng = random.Random(42)
        self.puzzle_cells = puzzle.cells.copy()
        self.is_fixed = puzzle.cells > 0
        
        initial_state, _ = initialize_grid(puzzle, rng)
        
        # Convertir en tuple pour simanneal (etat doit etre hashable)
        self.state_shape = initial_state.shape
        super().__init__(initial_state=tuple(initial_state.flatten()))
    
    def move(self):
        """Genere un voisin par echange."""
        state = np.array(self.state).reshape(self.state_shape)
        
        # Choisir une ligne avec au moins 2 cellules non fixes
        rows_with_free = [r for r in range(9) if np.sum(~self.is_fixed[r, :]) >= 2]
        if not rows_with_free:
            return
        
        row = random.choice(rows_with_free)
        free_cols = [c for c in range(9) if not self.is_fixed[row, c]]
        col1, col2 = random.sample(free_cols, 2)
        
        # Effectuer l'echange
        state[row, col1], state[row, col2] = state[row, col2], state[row, col1]
        
        self.state = tuple(state.flatten())
    
    def energy(self):
        """Calcule l'energie de l'etat actuel."""
        state = np.array(self.state).reshape(self.state_shape)
        return compute_energy(state)

print("Classe SudokuAnnealer definie.")

## 6. Test sur un Puzzle Facile

In [ ]:
print("=== Test : Puzzle Facile ===")
puzzle = SudokuGrid.from_string(easy_puzzles[0])
print("Puzzle original:")
print(puzzle)
print(f"Cellules vides: {np.sum(puzzle.cells == 0)}")

# Creer et executer le recuit simule
annealer = SudokuAnnealer(puzzle)
annealer.Tmax = 1.0      # Temperature initiale
annealer.Tmin = 0.001    # Temperature minimale
annealer.steps = 50000   # Nombre d'iterations
annealer.updates = 100   # Afficher tous les 100 pas

start = time.time()
state, e = annealer.anneal()
elapsed = time.time() - start

result = SudokuGrid(np.array(state).reshape(9, 9))
final_energy = compute_energy(result.cells)

print(f"\nSolution trouvee en {elapsed:.2f}s:")
print(result)
print(f"\nEnergie finale: {final_energy}")
print(f"Solution valide: {final_energy == 0}")

### Interpretation : Premier test

| Aspect | Observation |
|--------|-------------|
| Resultat | Le recuit simule peut trouver la solution pour les puzzles faciles |
| Temps | Plus lent que les solveurs deterministes |
| Non-determinisme | Chaque execution peut donner un resultat different |

> **Point cle** : contrairement au backtracking, le recuit simule n'est **pas garanti** de trouver la solution.

## 7. Implementation Manuelle du Recuit Simule

Pour mieux comprendre l'algorithme, implementons-le sans utiliser `simanneal`.

In [ ]:
class SimulatedAnnealingSolver:
    """Solveur de Sudoku par recuit simule (implementation manuelle)."""
    
    def __init__(self, T0: float = 1.0, alpha: float = 0.999, 
                 iterations_per_temp: int = 100, Tmin: float = 0.001):
        self.T0 = T0
        self.alpha = alpha
        self.iterations_per_temp = iterations_per_temp
        self.Tmin = Tmin
        self.energy_history = []
    
    def solve(self, puzzle: SudokuGrid, max_restarts: int = 5) -> Tuple[SudokuGrid, bool]:
        rng = random.Random(42)
        best_grid = None
        best_energy = float('inf')
        
        for restart in range(max_restarts):
            grid, is_fixed = initialize_grid(puzzle, rng)
            current_energy = compute_energy(grid)
            
            if current_energy < best_energy:
                best_energy = current_energy
                best_grid = SudokuGrid(grid.copy())
            
            if current_energy == 0:
                return best_grid, True
            
            T = self.T0
            total_iter = 0
            
            while T > self.Tmin and current_energy > 0:
                for _ in range(self.iterations_per_temp):
                    # Generer un voisin
                    row, col1, col2 = generate_neighbor(grid, is_fixed, rng)
                    new_energy = compute_energy(grid)
                    delta_E = new_energy - current_energy
                    
                    # Critere d'acceptation de Metropolis
                    if delta_E <= 0 or rng.random() < math.exp(-delta_E / T):
                        current_energy = new_energy
                        
                        if current_energy < best_energy:
                            best_energy = current_energy
                            best_grid = SudokuGrid(grid.copy())
                        
                        if current_energy == 0:
                            return best_grid, True
                    else:
                        # Rejeter : annuler l'echange
                        undo_swap(grid, row, col1, col2)
                    
                    total_iter += 1
                    
                    if total_iter % 500 == 0:
                        self.energy_history.append(current_energy)
                
                T *= self.alpha
            
            if best_energy == 0:
                return best_grid, True
        
        return best_grid, best_energy == 0

print("Classe SimulatedAnnealingSolver definie.")

In [ ]:
import math

# Test
print("=== Test : Recuit Simule Manuel ===")
puzzle = SudokuGrid.from_string(easy_puzzles[0])

solver = SimulatedAnnealingSolver(
    T0=1.0, alpha=0.999, 
    iterations_per_temp=100, Tmin=0.001
)

start = time.time()
result, solved = solver.solve(puzzle, max_restarts=10)
elapsed = time.time() - start

print(f"\nResolu: {solved}")
print(f"Temps: {elapsed:.2f}s")
print(f"\nSolution:")
print(result)

## 8. Benchmark sur Plusieurs Puzzles

In [ ]:
def benchmark_sa(puzzles: List[str], num_puzzles: int = 5):
    """Benchmark du recuit simule sur plusieurs puzzles."""
    print(f"\n=== Benchmark: Recuit Simule ({num_puzzles} puzzles) ===")
    
    results = []
    total_time = 0
    solved_count = 0
    
    for i, puzzle_str in enumerate(puzzles[:num_puzzles]):
        grid = SudokuGrid.from_string(puzzle_str)
        empty_count = np.sum(grid.cells == 0)
        
        solver = SimulatedAnnealingSolver(
            T0=1.0, alpha=0.999,
            iterations_per_temp=100, Tmin=0.001
        )
        
        start = time.time()
        result, solved = solver.solve(grid, max_restarts=10)
        elapsed = time.time() - start
        
        errors = compute_energy(result.cells)
        total_time += elapsed
        if solved:
            solved_count += 1
        
        status = "OK" if solved else f"{errors} erreurs"
        print(f"  Puzzle {i+1}: {status}, {empty_count} vides, {elapsed:.2f}s")
        results.append({'solved': solved, 'errors': errors, 'time': elapsed})
    
    print(f"\nResume:")
    print(f"  Resolus: {solved_count}/{num_puzzles}")
    print(f"  Temps total: {total_time:.2f}s")
    print(f"  Temps moyen: {total_time/num_puzzles:.2f}s")
    
    return results

benchmark_sa(easy_puzzles, num_puzzles=5)

### Interpretation : Resultats du benchmark

| Difficulte | Taux de succes attendu | Temps typique |
|------------|----------------------|---------------|
| Easy | 60-100% | 1-5s |
| Medium | 10-50% | 5-20s |
| Hard | < 10% | > 20s |

**Points cles** :
1. Le recuit simule **ne garantit pas** de trouver la solution
2. Les performances dependent fortement du reglage des parametres
3. Pour les puzzles difficiles, les solveurs CSP (OR-Tools, Z3) restent plus fiables

## 9. Comparaison avec Backtracking

In [ ]:
# Simple backtracking pour comparaison
class SimpleBacktracking:
    """Simple backtracking pour comparaison."""
    
    def __init__(self):
        self.call_count = 0
    
    def solve(self, puzzle: SudokuGrid) -> bool:
        grid = puzzle.cells.copy()
        self.call_count = 0
        return self._backtrack(grid)
    
    def _backtrack(self, grid: np.ndarray) -> bool:
        self.call_count += 1
        
        # Trouver case vide
        for r in range(9):
            for c in range(9):
                if grid[r, c] == 0:
                    for num in range(1, 10):
                        if self._is_valid(grid, r, c, num):
                            grid[r, c] = num
                            if self._backtrack(grid):
                                return True
                            grid[r, c] = 0
                    return False
        return True
    
    def _is_valid(self, grid: np.ndarray, row: int, col: int, num: int) -> bool:
        # Vérifier ligne
        if num in grid[row, :]:
            return False
        # Vérifier colonne
        if num in grid[:, col]:
            return False
        # Vérifier bloc
        br, bc = 3 * (row // 3), 3 * (col // 3)
        if num in grid[br:br+3, bc:bc+3]:
            return False
        return True

# Comparaison
print("=== Comparaison : Backtracking vs Recuit Simule ===")

for i, puzzle_str in enumerate(easy_puzzles[:3]):
    print(f"\nPuzzle {i+1}:")
    
    # Backtracking
    grid = SudokuGrid.from_string(puzzle_str)
    bt = SimpleBacktracking()
    start = time.time()
    bt.solve(grid)
    t_bt = time.time() - start
    print(f"  Backtracking: {bt.call_count} appels, {t_bt*1000:.1f} ms")
    
    # Recuit simule
    grid = SudokuGrid.from_string(puzzle_str)
    solver = SimulatedAnnealingSolver()
    start = time.time()
    result, solved = solver.solve(grid, max_restarts=5)
    t_sa = time.time() - start
    status = "OK" if solved else "Echec"
    print(f"  Recuit Simule: {status}, {t_sa*1000:.0f} ms")

## 10. Exercices

### Exercice 1 : Rechauffement

Implementez une version avec **rechauffement** : si l'energie ne diminue pas pendant N paliers, remonter la temperature.

### Exercice 2 : Parametres adaptatifs

Adaptez le taux de refroidissement `alpha` en fonction du taux d'acceptation des mouvements.

### Exercice 3 : Voisinage etendu

Experimentez avec d'autres types de mouvements : echanges entre lignes adjacentes, rotations de blocs, etc.

## Resume

### Concepts cles

| Concept | Description |
|---------|-------------|
| **Recuit simule** | Metaheuristique d'optimisation inspiree de la metallurgie |
| **Fonction d'energie** | Nombre de doublons colonnes + blocs |
| **Voisinage** | Echange de deux cellules non fixes dans une meme ligne |
| **Acceptation de Metropolis** | Accepter les degradations avec probabilite $e^{-\Delta E / T}$ |
| **Refroidissement** | Reduction progressive de $T$ (programme exponentiel) |

### Forces et limites

| Aspect | Avantage | Inconvenient |
|--------|----------|-------------|
| Garantie | Aucune | Ne trouve pas toujours la solution |
| Performance | Variable selon parametres | Plus lent que les solveurs CSP |
| Pedagogie | Elegant pour comprendre l'optimisation | Pas adapte aux CSP stricts |

### Quand l'utiliser

- Puzzles moyens (pas trop difficiles)
- Quand on veut comprendre les metaheuristiques
- Pour explorer l'espace de solutions

### Alternatives recommandees

- **Backtracking** : [Sudoku-1-Backtracking-Python](Sudoku-1-Backtracking-Python.ipynb)
- **OR-Tools CP-SAT** : [Sudoku-10-ORTools-Python](Sudoku-10-ORTools-Python.ipynb)
- **Z3 SMT** : [Sudoku-12-Z3-Python](Sudoku-12-Z3-Python.ipynb)

---

**Navigation** : [<< Sudoku-3-Genetic](Sudoku-3-Genetic-Python.ipynb) | [Index](README.md) | [Sudoku-5-PSO >>](Sudoku-5-PSO-Python.ipynb)